In [153]:
# disable compiler warnings
import os

# imports 
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Dense
from typing import List
import datetime
from tqdm.notebook import tqdm


from tensorflow.python.client import device_lib
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'  # FATAL
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))


Num GPUs Available:  []


In [154]:
(train_ds, val_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

#tfds.show_examples(train_ds, ds_info)

In [155]:
class BasicConv(tf.keras.Model):
    def __init__(self, optimizer=tf.keras.optimizers.Adam()):
        super(BasicConv, self).__init__()

        self.optimizer = optimizer
        self.loss_function = tf.keras.losses.CategoricalCrossentropy()
        self.metrics_list = [[tf.keras.metrics.CategoricalAccuracy(name="test_accuracy"),
                             tf.keras.metrics.Mean(name="test_loss"),
                             tf.keras.metrics.Mean(name="test_frob_norm")],
                             [tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
                             tf.keras.metrics.Mean(name="train_loss"),
                             tf.keras.metrics.Mean(name="train_frob_norm")]]
  
        self.pooling = tf.keras.layers.MaxPooling2D()
        self.my_layers = [
                        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(28, 28, 1)),
                        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(28, 28, 1)),
                        tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D()),
                        tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(14, 14, 1)),
                        tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(14, 14, 1)),
                        tf.keras.layers.TimeDistributed(tf.keras.layers.GlobalAvgPool2D()),
                        tf.keras.layers.Dense(10, activation='softmax')
                        ]



    @tf.function
    def call(self, x, training=False):
        for layer in self.my_layers:
            tf.print(x)
            x = layer(x)
        return x

    @tf.function
    def compute_frobenius(self):
        frobenius_norm = tf.zeros((1,))
        for var in self.trainable_variables:
            frobenius_norm += tf.norm(var, ord="euclidean")
        return frobenius_norm

    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list

    # 4. reset all metrics objects
    def reset_metrics(self, num):
        for metric in self.metrics[num]:
            metric.reset_states()


    # train_step method
    @tf.function
    def train_step(self, data):
        img, label = data
        
        # compute output and loss, train the variables
        with tf.GradientTape() as tape:
            output = self(img, training=True)
            loss = self.loss_function(label, output)
            
        # update trainable variables
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # update metrics
        self.metrics_list[1][0].update_state(tf.argmax(output, axis=1), tf.argmax(label, axis=1))
        self.metrics_list[1][1].update_state(loss)
        self.metrics_list[1][2].update_state(self.compute_frobenius())
        
        # return a dict with metric information
        return {m.name : m.result() for m in self.metrics_list[1]}



    # test_step method
    @tf.function
    def test_step(self, data):
        img, label = data

        # compute output and loss, without training
        output = self(img, training=False)
        loss = self.loss_function(label, output)

        # update metrics
        self.metrics_list[0][0].update_state(tf.argmax(output, axis=1), tf.argmax(label, axis=1))
        self.metrics_list[0][1].update_state(loss)
        self.metrics_list[0][2].update_state(self.compute_frobenius())

        # return a dict with metric information 
        return {m.name : m.result() for m in self.metrics_list[0]}

In [ ]:
class CustomLSTM(tf.keras.layers.AbstractRNNCell):
  def __init__(self, units, **kwargs):
      self.units = units
      super(CustomLSTM, self).__init__(**kwargs)

  @property
  def state_size(self):
    return self.units

  @property
  def output_size(self):
    return 1

  @property
  def get_initial_state(self):
    return tf.zeros(shape=(1, 10)) # TODO

#### TODO ####
  def build(self, input_shape):
    self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                  initializer='uniform',
                                  name='kernel')
    self.recurrent_kernel = self.add_weight(
        shape=(self.units, self.units),
        initializer='uniform',
        name='recurrent_kernel')
    self.built = True

  def call(self, inputs, states):
    prev_output = states[0]
    h = backend.dot(inputs, self.kernel)
    output = h + backend.dot(prev_output, self.recurrent_kernel)
    return output, output
##########

In [156]:

def create_summary_writers(config_name):
    
    # Define where to save the logs
    # along with this, you may want to save a config file with the same name so you know what the hyperparameters were used
    # alternatively make a copy of the code that is used for later reference
    
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/{config_name}/{current_time}/train"
    val_log_path = f"logs/{config_name}/{current_time}/val"

    # log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)

    # log writer for validation metrics
    val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
    return train_summary_writer, val_summary_writer

In [157]:
# gets in a dataset and returns target values
def prepare_data(dataset, seq_size, batch_size):

    # get list of all targets
    targets = []
    for images, labels in dataset:  # only take first element of dataset
        targets.append(labels.numpy())

    # convert data from uint8 to float32
    dataset = dataset.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # convert image values from range [0, 255] to [-1, 1]
    dataset = dataset.map(lambda img, target: ((img/128.)-1., target))

    # Create random tuples of 4 images
    dataset = dataset.shuffle(4096).window(seq_size, drop_remainder=True)

    # Alternate signs
    def scan_alternating_sign(sign, elem):
      return sign*-1, elem*sign

    dataset = dataset.map(lambda img_ds, target_ds: (img_ds, target_ds.scan(initial_state=tf.constant(1, dtype=tf.int64), scan_func=scan_alternating_sign)))

    # Sum targets
    def scan_target_sum(state, elem):
      sum = state + elem
      return sum, sum

    dataset = dataset.map(lambda img_ds, target_ds: (img_ds, target_ds.scan(initial_state=tf.constant(0, dtype=tf.int64), scan_func=scan_target_sum)))

    dataset = dataset.cache()
    dataset = dataset.shuffle(4096)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [158]:
# trains the model by iterating through the dataset and applying training_step method epochs time
def training_loop(model, train_ds, test_ds, epochs, train_summary_writer, memory):
    metrics = []

    # iterate over epochs
    for epoch in tqdm(range(epochs)):

        # train steps on all batches in the training data
        for (img, label) in train_ds:
            metrics = model.train_step((img, label))
            
            # keep data in summary with metrics
            with train_summary_writer.as_default():
                for metric in model.metrics_list[1]:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        for (key, value) in metrics.items():
            memory[key].append(value.numpy())

        memory = test_loop(model=model,
                           test_ds=test_ds,
                           val_summary_writer=val_summary_writer,
                           memory=memory)
        
        # print current metric values and reset the metrics
        tf.print([f"{key} : {value.numpy()}" for (key, value ) in metrics.items()])
        model.reset_metrics(1)

    return memory


# tests overall performance of model
def test_loop(model, test_ds, val_summary_writer, memory):
    metrics = []
    # test steps on every item in test dataset
    for (img, label) in tqdm(test_ds):
        metrics = model.test_step((img, label))
        
        # keep data with metrics
        with val_summary_writer.as_default():
            for metric in model.metrics_list[0]:
                tf.summary.scalar(f"{metric.name}", metric.result(), step=1)

    for (key, value) in metrics.items():
        memory[key].append(value.numpy())

    print([f"{key} : {value.numpy()}" for (key, value ) in metrics.items()])

    model.reset_metrics(0)

    return memory

# visualize accuracy, loss and frobenius norm
def visualization(accuracies, losses, frobenius, name):
    plt.figure()
    line1, = plt.plot(accuracies, "b")
    line2, = plt.plot(losses, "r")

    frob_new = frobenius/np.max(frobenius) * np.max(losses)
    line3, = plt.plot(frob_new, "y" )

    plt.xlabel("Training steps")
    plt.ylabel("Loss/Accuracy")
    plt.legend((line1, line2, line3),("Accuracy", "Loss", "Frobenius Norm"))
    plt.savefig(name)
    plt.show()


In [159]:
# overall hyperparameters to compare with and without overfitting precautions methods
epochs = 15
batch_size = 32
seq_size = 4

## no augmentation, normal model

In [162]:
train_summary_writer, val_summary_writer = create_summary_writers(config_name="UNTOUCHED")

model = BasicConv()


for img, label in train_ds.take(1):
    print(img.shape)
    print(label.shape)

#train_dataset = prepare_data(train_ds, seq_size, batch_size)
train_dataset = train_ds.apply(lambda dataset: prepare_data(dataset, seq_size, batch_size))
val_dataset = prepare_data(val_ds, seq_size, batch_size)

#for img, label in train_dataset.take(1):
#    print(img.shape)
#    print(label.shape)

(28, 28, 1)
()


In [161]:
tfds.benchmark(train_dataset, batch_size=batch_size)

print("\n************ Training UNTOUCHED ************\n")

memory = {"test_accuracy" : [],
          "test_loss" : [],
          "test_frob_norm" : [],
          "train_accuracy" : [],
          "train_loss" : [],
          "train_frob_norm" : []
          }

memory = training_loop(model,
                       train_ds=train_dataset,
                       test_ds=val_dataset,
                       epochs=tf.constant(epochs),
                       train_summary_writer=train_summary_writer,
                       memory=memory)

visualization(memory["train_accuracy"], memory["train_loss"], memory["train_frob_norm"], "aug_train")

  0%|          | 1/469 [00:00<?, ?it/s]


************ Summary ************

Examples/sec (First included) 2140.93 ex/sec (total: 15008 ex, 7.01 sec)
Examples/sec (First only) 20.16 ex/sec (total: 32 ex, 1.59 sec)
Examples/sec (First excluded) 2761.76 ex/sec (total: 14976 ex, 5.42 sec)

************ Training UNTOUCHED ************



  0%|          | 0/15 [00:00<?, ?it/s]

TypeError: ignored